# Imports e config dos módulos

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
import warnings
from sklearn.feature_selection import f_classif, mutual_info_classif, SequentialFeatureSelector, SelectKBest
from scipy.stats import chi2_contingency
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict, LeaveOneOut, cross_val_score, StratifiedKFold, GridSearchCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from boruta import BorutaPy
from genetic_selection import GeneticSelectionCV

In [16]:
# pd.set_option('max_columns', None)
%matplotlib inline
sns.set_style('darkgrid')

# Dataset

In [17]:
df = pd.read_csv('../data/interim/hans_outcome.csv')

C:\Users\mclar\AppData\Local\Temp\ipykernel_17292\1568157318.py:1: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/interim/hans_outcome.csv')


In [18]:
df.shape

(113837, 39)

# Agrupamento e remoção de features

In [19]:
target = 'TPALTA_N'
feats = df.drop(target, axis=1).columns
num_feats = [feat for feat in feats if df[feat].dtype != 'O']
cat_feats = [feat for feat in feats if feat not in num_feats]

In [20]:
# remove colunas com classes muito dominantes
def drop_low_variance(col):
    if df[col].nunique() == 1:
        return True
    elif df[col].value_counts(1).iloc[0] > 0.9:
        return True
    elif df[col].value_counts().iloc[0] < df[col].isna().sum():
        return True
    else:
        return False

low_var_feats = [col for col in cat_feats if drop_low_variance(col)]
cat_feats = [feat for feat in cat_feats if feat not in low_var_feats]

In [21]:
cat_feats

['DT_DIAG',
 'CS_SEXO',
 'CS_ESCOL_N',
 'DTINICTRAT',
 'UFATUAL',
 'DT_NOTI_AT',
 'DTALTA_N']

# ML Pipeline

## Feature transformers

In [22]:
class GBMFeatTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, EBM=False):
        self.EBM = EBM
        self.num_feats_means = dict()
        
    def fit(self, X, y=None):
        if self.EBM:
            for col in X.columns:
                if X[col].dtype != 'O':
                    self.num_feats_means[col] = X[col].mean()
        return self
            
    def transform(self, X, y=None):
        ret = X.copy()
        if not self.EBM:
            for col in ret.columns:
                if ret[col].dtype == 'O':
                    ret[col] = ret[col].astype('category')
        else:
            for col in self.num_feats_means:
                ret[col] = ret[col].fillna(self.num_feats_means[col])
        return ret
        
general_num_transformer = FeatureUnion(    
    [
        ('num_pipe', Pipeline(
            [
                ('norm', StandardScaler()),
                ('nan_input', SimpleImputer())
            ]
        )),
        ('nan_flag', MissingIndicator(error_on_new=False))
    ]
)

general_feat_transformer = ColumnTransformer(
    [
        ('num_trans', general_num_transformer, num_feats),
        ('cat_trans', OneHotEncoder(handle_unknown='ignore'), cat_feats)    
    ],
    remainder='passthrough', sparse_threshold=0
)

## Nested K-Fold

In [23]:
class NestedKFoldOpt():
    def __init__(self, ml_model, opt_space, loss_metric, outer_cv, inner_cv, 
                 opt_type='bayes', max_evals=10):
        self.ml_model = ml_model
        self.opt_space = opt_space
        self.loss_metric = loss_metric
        self.opt_type = opt_type
        self.max_evals = max_evals
        self.outer_cv = outer_cv
        self.inner_cv = inner_cv
        self.metrics_ = None
        self.metrics_oof_ = None
        self.metrics_dist_ = None
        self.best_hyperparameters_ = None
        
    def objective(self, x, data):
        model = clone(self.ml_model).set_params(**x)
        
        preds = cross_val_predict(model, data[0], data[1], cv=self.inner_cv, n_jobs=-1)
        
        return -self.loss_metric(data[1], preds)     
        
    
    def optimize(self, X, y):
        if self.opt_type == 'bayes':
            obj = partial(self.objective, data=(X, y))
            best = fmin(obj, space=self.opt_space, algo=tpe.suggest, 
                        max_evals=self.max_evals, return_argmin=False)
        else:
            loss_metric = make_scorer(self.loss_metric)
            best = GridSearchCV(self.ml_model, self.opt_space, scoring=loss_metric,
                                n_jobs=-1, cv=self.inner_cv, verbose=3).\
                   fit(X, y).best_params_
        return best
    
    def nested_kfold(self, X, y):
        recall_0 = []
        recall_1 = []
        precision_0 = []
        precision_1 = []
        oof = np.zeros(len(X))
        for train_idx, val_idx in self.outer_cv.split(X, y):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
            hypers = self.optimize(X_train, y_train)
            model = clone(self.ml_model).set_params(**hypers).fit(X_train, y_train)
            preds = model.predict(X_val)
            oof[val_idx] = preds 
            recall_0.append(recall_score(y_val, preds, pos_label=0))
            recall_1.append(recall_score(y_val, preds, pos_label=1))
            precision_0.append(precision_score(y_val, preds, pos_label=0))
            precision_1.append(precision_score(y_val, preds, pos_label=1))
        self.metrics_ = pd.DataFrame({'recall': [np.mean(recall_0), np.mean(recall_1)],
                                      'recall_std': [np.std(recall_0), np.std(recall_1)],
                                      'precision': [np.mean(precision_0), np.mean(precision_1)],
                                      'precision_std': [np.std(precision_0), np.std(precision_1)]},
                                     index=['class_0', 'class_1'])
        self.metrics_oof_ = pd.DataFrame({'recall': [recall_score(y, oof, pos_label=0), recall_score(y, oof)],
                                          'precision': [precision_score(y, oof, pos_label=0), precision_score(y, oof)],
                                          'f1 score': [f1_score(y, oof, pos_label=0), f1_score(y, oof)],
                                          'accuracy': [accuracy_score(y, oof)],
                                         },
                                         index=['class_0', 'class_1'])
        self.metrics_dist_ = {'recall_0': recall_0, 'recall_1': recall_1,
                              'precision_0': precision_0, 'precision_1': precision_1}
            
    def fit(self, X, y):
        X = X.copy()
        y = y.copy()
        self.nested_kfold(X, y)
        self.best_hyperparameters_ = self.optimize(X, y)

In [24]:
X = df[num_feats + cat_feats].copy()
le = LabelEncoder().fit(df[target])
y = pd.Series(le.transform(df[target]))

## Feature Selection

In [25]:
base_model = RandomForestClassifier(n_jobs=-1, max_depth=5)

In [26]:
X_num = GBMFeatTransformer().fit_transform(X)
for col in X_num.columns:
    if X_num[col].dtype.name == 'category':
        X_num[col] = X_num[col].cat.codes
    else:
        X_num[col].fillna(X_num[col].median(), inplace=True)
        
columns = X_num.columns.values

In [27]:
print(classification_report(y, cross_val_predict(base_model, X_num, y, n_jobs=-1,
  cv=StratifiedKFold(shuffle=True, random_state=9))))

              precision    recall  f1-score   support

           0       0.93      0.04      0.08      6880
           1       0.94      1.00      0.97    106957

    accuracy                           0.94    113837
   macro avg       0.94      0.52      0.52    113837
weighted avg       0.94      0.94      0.92    113837



### Boruta

In [28]:
bor_selector = BorutaPy(base_model, n_estimators='auto', perc=90, max_iter=100)
bor_selector.fit(X_num.values, y.values)

In [ ]:
columns[bor_selector.support_]

array(['SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'ANO_NASC',
       'CS_RACA', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'FORMACLINI',
       'AVALIA_N', 'CLASSOPERA', 'MODODETECT', 'BACILOSCOP', 'ESQ_INI_N',
       'ID_MUNI_AT', 'ID_UNID_AT', 'UFRESAT', 'MUNIRESAT', 'CLASSATUAL',
       'AVAL_ATU_N', 'ESQ_ATU_N', 'EPIS_RACIO', 'NU_ANO', 'NU_LESOES',
       'CONTREG', 'DOSE_RECEB', 'CONTEXAM', 'DURACAO_TRAT', 'DT_DIAG',
       'DTINICTRAT', 'UFATUAL', 'DT_NOTI_AT', 'DTALTA_N'], dtype=object)

In [ ]:
bottom_boruta = columns[bor_selector.ranking_.argsort()[-(len(columns)-30):]]
bottom_boruta

array(['MODODETECT', 'BACILOSCOP', 'ID_MN_RESI', 'DTALTA_N', 'CS_GESTANT',
       'CS_ESCOL_N', 'CS_SEXO'], dtype=object)

### Sequential Selector

In [ ]:
sb_selector = SequentialFeatureSelector(base_model, n_features_to_select=30, direction='backward',
                                        scoring=make_scorer(f1_score, average='macro'), 
                                        cv=StratifiedKFold(shuffle=True, random_state=9), n_jobs=-1)
sb_selector.fit(X_num, y)

In [ ]:
bottom_sb = columns[~sb_selector.get_support()]
bottom_sb

### M.I Filtering

In [ ]:
mi_selector = SelectKBest(mutual_info_classif, k=30).fit(X_num, y)

In [ ]:
bottom_mi = columns[~mi_selector.get_support()]
bottom_mi

Out[29]: array(['sopi', 'pdv_age', 'rewards_balance', 'rewards_adoption',
 'late_request', 'open_request', 'request_nok_l15d', 'days_overdue',
 'volume_nab_l90d', 'volume_beer_l90d', 'volume_match_l30d',
 'avg_suggested_order_l30d', 'total_nfe_l90d', 'rewards_usage_l90d',
 'faturamento_mktplace_l90d', 'faturamento_mktplace_l30d',
 'count_consecutivemonths_purchase_beer_l3m',
 'count_consecutivemonths_purchase_nab_l3m',
 'count_consecutivemonths_purchase_rgb_l3m',
 'total_skus_mktplace_l30d', 'total_skus_l30d',
 'total_skus_beer_l30d', 'total_skus_match_l30d',
 'total_skus_rgb_l30d', 'devolucao_l30d', 'devolucao_l90d',
 'peso_nab_l30d', 'cash_payment_amount_l30d',
 'is_credit_card_pos_payer', 'is_check_payer',
 'is_direct_pay_at_delivery_payer', 'is_bank_slip_payer', 'cod_unb',
 'tipo_documento', 'documentacao_ok'], dtype=object)

### Genetic Algorithm

In [ ]:
ga_selector = GeneticSelectionCV(base_model, cv=StratifiedKFold(shuffle=True, random_state=9), 
                                 scoring=make_scorer(f1_score, average='macro'), max_features=30, 
                                 n_population=100, n_generations=40, n_gen_no_change=10)
ga_selector.fit(X_num, y)

In [ ]:
bottom_ga = columns[~ga_selector.get_support()]
bottom_ga

### Aggregation of the methods

In [ ]:
counts = pd.Series(dict(Counter(np.hstack([bottom_boruta, bottom_sb, bottom_mi, bottom_ga]))))
# counts = pd.Series(dict(Counter(np.hstack([bottom_boruta, bottom_mi]))))

In [ ]:
cols_to_decide = counts[counts>2].index
cols_to_decide

Out[33]: Index(['peso_nab_l30d', 'cash_payment_amount_l30d', 'total_skus_mktplace_l30d',
 'total_skus_match_l30d', 'avg_suggested_order_l30d',
 'is_bank_slip_payer', 'rewards_usage_l90d',
 'count_consecutivemonths_purchase_rgb_l3m',
 'count_consecutivemonths_purchase_nab_l3m',
 'count_consecutivemonths_purchase_beer_l3m', 'sopi',
 'is_direct_pay_at_delivery_payer', 'documentacao_ok', 'devolucao_l90d',
 'devolucao_l30d', 'days_overdue', 'rewards_adoption', 'is_check_payer',
 'tipo_documento', 'open_request', 'request_nok_l15d',
 'is_credit_card_pos_payer', 'late_request'],
 dtype='object')

In [ ]:
cols_to_remove = cols_to_decide

In [ ]:
X_fs = X.drop(cols_to_remove, axis=1)
X_fs.shape

Out[14]: (10320, 21)

## Algorithms Comparison

### Logistic Regression

In [ ]:
lr_pipe = Pipeline(
  [
      ('feat_trans', general_feat_transformer),
      ('over', RandomOverSampler(random_state=9)),
      ('logreg', LogisticRegression(random_state = 0))
]
)

lr_opt_space = {'logreg__solver': hp.choice('logreg__solver', ['liblinear', 'lbfgs']),
                'logreg__C': hp.loguniform('logreg__C', np.log(1e-5), np.log(100))}
               
lr_opt_space = {'logreg__warm_start' : hp.choice('logreg__warm_start', [True, False]),
                'logreg__fit_intercept' : hp.choice('logreg__fit_intercept', [True, False]),
                'logreg__tol' : hp.uniform('logreg__tol', 0.00001, 0.0001),
                'logreg__C' : hp.uniform('logreg__C', 0.05, 3),
                'logreg__solver' : hp.choice('logreg__solver', ['newton-cg', 'lbfgs', 'liblinear']),
                'logreg__multi_class' : 'auto',
                'logreg__class_weight' : 'balanced'}

lr_opt = NestedKFoldOpt(lr_pipe, lr_opt_space, partial(f1_score, average='macro'), 
                      outer_cv=StratifiedKFold(2, shuffle=True, random_state=9), 
                      inner_cv=StratifiedKFold(2, shuffle=True, random_state=9))

NameError: name 'Pipeline' is not defined

In [ ]:
lr_opt.fit(X, y)

0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [00:01<00:17, 1.99s/trial, best loss: -0.7255894064747616] 20%|██ | 2/10 [00:07<00:33, 4.19s/trial, best loss: -0.7255894064747616] 30%|███ | 3/10 [00:11<00:26, 3.83s/trial, best loss: -0.726914913320065]  40%|████ | 4/10 [00:12<00:16, 2.71s/trial, best loss: -0.7317316308243726] 50%|█████ | 5/10 [00:12<00:10, 2.04s/trial, best loss: -0.7317316308243726] 60%|██████ | 6/10 [00:15<00:09, 2.33s/trial, best loss: -0.7317316308243726] 70%|███████ | 7/10 [00:16<00:05, 1.90s/trial, best loss: -0.7317316308243726] 80%|████████ | 8/10 [00:20<00:04, 2.32s/trial, best loss: -0.7317316308243726] 90%|█████████ | 9/10 [00:23<00:02, 2.76s/trial, best loss: -0.7317316308243726]100%|██████████| 10/10 [00:27<00:00, 3.17s/trial, best loss: -0.7317316308243726]100%|██████████| 10/10 [00:27<00:00, 2.79s/trial, best loss: -0.7317316308243726]
/databricks/python/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
 https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
 https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 n_iter_i = _check_optimize_result(
 0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [00:00<00:08, 1.03trial/s, best loss: -0.713739765470829] 20%|██ | 2/10 [00:01<00:07, 1.02trial/s, best loss: -0.713739765470829] 30%|███ | 3/10 [00:02<00:06, 1.14trial/s, best loss: -0.7230187703902595] 40%|████ | 4/10 [00:03<00:05, 1.01trial/s, best loss: -0.7230187703902595] 50%|█████ | 5/10 [00:07<00:10, 2.07s/trial, best loss: -0.7231309982587271] 60%|██████ | 6/10 [00:08<00:06, 1.70s/trial, best loss: -0.7278385274438908] 70%|███████ | 7/10 [00:11<00:05, 1.87s/trial, best loss: -0.7317377462126882] 80%|████████ | 8/10 [00:15<00:05, 2.65s/trial, best loss: -0.7317377462126882] 90%|█████████ | 9/10 [00:16<00:02, 2.13s/trial, best loss: -0.7317377462126882]100%|██████████| 10/10 [00:22<00:00, 3.25s/trial, best loss: -0.7317377462126882]100%|██████████| 10/10 [00:22<00:00, 2.21s/trial, best loss: -0.7317377462126882]
 0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [00:02<00:22, 2.54s/trial, best loss: -0.7244330956828009] 20%|██ | 2/10 [00:05<00:20, 2.53s/trial, best loss: -0.7244330956828009] 30%|███ | 3/10 [00:07<00:17, 2.50s/trial, best loss: -0.7251628060360091] 40%|████ | 4/10 [00:26<00:53, 8.97s/trial, best loss: -0.7257314315103411] 50%|█████ | 5/10 [00:40<00:54, 10.83s/trial, best loss: -0.7257314315103411] 60%|██████ | 6/10 [00:42<00:31, 7.96s/trial, best loss: -0.7264336917562724] 70%|███████ | 7/10 [00:44<00:17, 5.92s/trial, best loss: -0.7314812452947539] 80%|████████ | 8/10 [01:02<00:19, 9.65s/trial, best loss: -0.7314812452947539] 90%|█████████ | 9/10 [01:04<00:07, 7.37s/trial, best loss: -0.7314812452947539]100%|██████████| 10/10 [01:07<00:00, 5.83s/trial, best loss: -0.7314812452947539]100%|██████████| 10/10 [01:07<00:00, 6.70s/trial, best loss: -0.7314812452947539]

In [ ]:
lr_opt.metrics_oof_ 

Out[40]:

,recall,precision,f1 score,accuracy
class_0,0.712945,0.551589,0.621972,0.785271
class_1,0.809094,0.895367,0.850047,0.785271


In [ ]:
lr_opt.fit(X_fs, y)

0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [00:00<00:07, 1.15trial/s, best loss: -0.7383172286051909] 20%|██ | 2/10 [00:01<00:05, 1.47trial/s, best loss: -0.7391495149475011] 40%|████ | 4/10 [00:01<00:01, 3.33trial/s, best loss: -0.7399823446140972] 60%|██████ | 6/10 [00:01<00:00, 5.36trial/s, best loss: -0.7399823446140972] 80%|████████ | 8/10 [00:01<00:00, 7.16trial/s, best loss: -0.7399823446140972]100%|██████████| 10/10 [00:01<00:00, 8.82trial/s, best loss: -0.7399823446140972]100%|██████████| 10/10 [00:01<00:00, 5.10trial/s, best loss: -0.7399823446140972]
 0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 20%|██ | 2/10 [00:00<00:00, 13.82trial/s, best loss: -0.734619170952832] 40%|████ | 4/10 [00:00<00:00, 14.83trial/s, best loss: -0.734619170952832] 60%|██████ | 6/10 [00:00<00:00, 16.79trial/s, best loss: -0.734619170952832] 80%|████████ | 8/10 [00:00<00:00, 17.15trial/s, best loss: -0.734619170952832]100%|██████████| 10/10 [00:00<00:00, 14.09trial/s, best loss: -0.734619170952832]100%|██████████| 10/10 [00:00<00:00, 14.86trial/s, best loss: -0.734619170952832]
 0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [00:00<00:01, 4.91trial/s, best loss: -0.7360782931148023] 20%|██ | 2/10 [00:00<00:01, 4.40trial/s, best loss: -0.7360782931148023] 30%|███ | 3/10 [00:00<00:01, 4.72trial/s, best loss: -0.7360782931148023] 40%|████ | 4/10 [00:00<00:01, 4.56trial/s, best loss: -0.7360782931148023] 50%|█████ | 5/10 [00:01<00:01, 4.74trial/s, best loss: -0.7364435129529089] 60%|██████ | 6/10 [00:01<00:00, 5.03trial/s, best loss: -0.7364435129529089] 70%|███████ | 7/10 [00:01<00:00, 5.04trial/s, best loss: -0.7364435129529089] 80%|████████ | 8/10 [00:01<00:00, 5.08trial/s, best loss: -0.7364435129529089] 90%|█████████ | 9/10 [00:01<00:00, 4.94trial/s, best loss: -0.7364435129529089]100%|██████████| 10/10 [00:02<00:00, 5.15trial/s, best loss: -0.7364435129529089]100%|██████████| 10/10 [00:02<00:00, 4.94trial/s, best loss: -0.7364435129529089]

In [ ]:
lr_opt.metrics_oof_

Out[50]:

,recall,precision,f1 score,accuracy
class_0,0.746578,0.541100,0.627445,0.780329
class_1,0.791447,0.904594,0.844246,0.780329


### Logistic Regression

In [ ]:
rf_pipe = Pipeline(
    [
        ('feat_trans', GBMFeatTransformer(high_card_feats_fs)),
        ('over', RandomOverSampler(random_state=9)),
        ('rf', LGBMClassifier(boosting_type='rf', subsample_freq=1, min_child_samples=1))
    ]
)

rf_opt_space = {'rf__n_estimators': scope.int(hp.quniform('rf__n_estimators', 50, 300, 10)),
                'rf__num_leaves': scope.int(hp.quniform('num_leaves', 2, 100, 1)),
                'rf__subsample': hp.uniform('rf__subsample', 0.3, 0.95),
                'rf__colsample_bytree': hp.uniform('rf__colsample_bytree', 0.3, 0.95)}

rf_opt = NestedKFoldOpt(rf_pipe, rf_opt_space, partial(f1_score, average='macro'), 
                        outer_cv=StratifiedKFold(10, shuffle=True, random_state=9), 
                        inner_cv=StratifiedKFold(10, shuffle=True, random_state=9))

In [ ]:
rf_opt.fit(X_fs, y)

In [ ]:
rf_opt.metrics_

In [ ]:
rf_opt.fit(X, y)

In [ ]:
rf_opt.metrics_

### SVM

svm_pipe = Pipeline(
    [
        ('feat_trans', general_feat_transformer),
        ('over', RandomOverSampler(random_state=9)),
        ('svm', SVC())
    ]
)

svm_opt_space = {'svm__C': hp.loguniform('svm__C', np.log(1e-5), np.log(100)),
                 'svm__gamma': hp.loguniform('svm__gamma', np.log(1e-6), np.log(10))}

svm_opt = NestedKFoldOpt(svm_pipe, svm_opt_space, partial(f1_score, average='macro'), 
                        outer_cv=StratifiedKFold(10, shuffle=True, random_state=9), 
                        inner_cv=StratifiedKFold(10, shuffle=True, random_state=9))

In [ ]:
svm_opt.fit(X, y)

In [ ]:
svm_opt.metrics_

### SGDClassifier

sgd_pipe = Pipeline(
    [
        ('feat_trans', general_feat_transformer),
        ('over', RandomOverSampler(random_state=9)),
        ('sgd', SGDClassifier())
    ]
)

sgd_opt_space = {'sgd__loss': hp.choice('sgd__loss', ['hinge', 'log']),
                 'sgd__alpha': hp.loguniform('sgd__alpha', np.log(1e-5), np.log(10)), 
                 'sgd__max_iter': scope.int(hp.quniform('sgd__max_iter', 10, 500, 10))}

sgd_opt = NestedKFoldOpt(sgd_pipe, sgd_opt_space, partial(f1_score, average='macro'), 
                        outer_cv=StratifiedKFold(10, shuffle=True, random_state=9), 
                        inner_cv=StratifiedKFold(10, shuffle=True, random_state=9))

In [ ]:
sgd_opt.fit(X, y)

In [ ]:
sgd_opt.metrics_

### LGBM

lgbm_pipe = Pipeline(
    [
        ('feat_trans', GBMFeatTransformer(high_card_feats_fs)),
        ('over', RandomOverSampler(random_state=9)),
        ('lgbm', LGBMClassifier())
    ]
)

lgbm_opt_space = {'lgbm__learning_rate': hp.loguniform('lgbm__learning_rate', np.log(0.001), np.log(0.5)),
                  'lgbm__reg_alpha': hp.loguniform('lgbm__reg_alpha', np.log(0.001), np.log(1)),
                  'lgbm__reg_lambda': hp.loguniform('lgbm__reg_lambda', np.log(0.001), np.log(1)),
                  'lgbm__subsample': hp.uniform('lgbm__subsample', 0.2, 1),
                  'lgbm__colsample_bytree': hp.uniform('lgbm__colsample_bytree', 0.2, 1),
                  'lgbm__min_child_samples': scope.int(hp.quniform('lgbm__min_child_samples', 1, 100, 1)),
                  'lgbm__num_leaves': scope.int(hp.quniform('lgbm__num_leaves', 2, 50, 1)),
                  'lgbm__subsample_freq': scope.int(hp.quniform('lgbm__subsample_freq', 1, 10, 1)),
                  'lgbm__n_estimators': scope.int(hp.quniform('lgbm__n_estimators', 100, 5000, 1))}

lgbm_opt = NestedKFoldOpt(lgbm_pipe, lgbm_opt_space, partial(f1_score, average='macro'), 
                        outer_cv=StratifiedKFold(10, shuffle=True, random_state=9), 
                        inner_cv=StratifiedKFold(10, shuffle=True, random_state=9))

In [ ]:
lgbm_opt.fit(X_fs, y)

In [ ]:
lgbm_opt.metrics_

In [ ]:
lgbm_opt.fit(X, y)

In [ ]:
lgbm_opt.metrics_

### EBM

In [ ]:
ebm_pipe = Pipeline(
    [
        ('feat_trans', GBMFeatTransformer(high_card_feats_fs, EBM=True)),
        #('over', RandomOverSampler(random_state=9)),
        ('ebm', ExplainableBoostingClassifier(n_jobs=1, validation_size=0))
    ]
)

ebm_opt_space = {'ebm__learning_rate': hp.loguniform('ebm__learning_rate', np.log(0.001), np.log(0.5)),
                 #'ebm__validation_size': hp.uniform('ebm__validation_size', 0.05, 0.25),
                 #'ebm__early_stopping_rounds': scope.int(hp.quniform('early__stopping_rounds', 5, 100, 1)),
                 'ebm__max_rounds': scope.int(hp.quniform('ebm__max_rounds', 10, 3000, 1)),
                 'ebm__interactions': scope.int(hp.quniform('ebm__interactions', 0, 20, 1)),
                 'ebm__max_leaves': scope.int(hp.quniform('ebm__max_leaves', 2, 10, 1)),
                 'ebm__outer_bags': scope.int(hp.quniform('ebm__outer_bags', 8, 16, 1)),
                 #'ebm__inner_bags': scope.int(hp.quniform('ebm__inner_bags', 0, 5, 1)),
                 'ebm__max_bins': scope.int(hp.quniform('ebm__max_bins', 8, 128, 1)),
                 'ebm__max_interaction_bins': scope.int(hp.quniform('ebm__max_interaction_bins', 8, 64, 1)),
                 'ebm__min_samples_leaf': scope.int(hp.quniform('ebm__min_samples_leaf', 1, 30, 1))}

ebm_opt = NestedKFoldOpt(ebm_pipe, ebm_opt_space, partial(f1_score, average='macro'), 
                        outer_cv=StratifiedKFold(10, shuffle=True, random_state=9), 
                        inner_cv=StratifiedKFold(10, shuffle=True, random_state=9))

In [ ]:
ebm_opt.fit(X_fs, y)

In [ ]:
ebm_opt.metrics_

In [ ]:
ebm_opt.fit(X, y)

In [ ]:
ebm_opt.metrics_

### Comparison

In [ ]:
df_metrics = pd.DataFrame()
for a, m in zip(['rf', 'svm', 'sgd', 'lgbm', 'ebm'], [rf_opt.metrics_dist_, svm_opt.metrics_dist_, 
                                                      sgd_opt.metrics_dist_, lgbm_opt.metrics_dist_, 
                                                      ebm_opt.metrics_dist_]):
    m['algo'] = [a]*len(m['recall_0'])
    df_metrics = pd.concat([df_metrics, pd.DataFrame(m)])

In [ ]:
make_subplots(df_metrics, df_metrics.columns[:-1], 'algo')

In [ ]:
# Dummy
print(classification_report(y, DummyClassifier(strategy='stratified').fit(X, y).predict(X)))